In [1]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.llms import OpenAIChat
from langchain.prompts import PromptTemplate

import chromadb

In [65]:

raw_documents = PyPDFDirectoryLoader('assets/').load()
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
db = Chroma.from_documents(documents, OpenAIEmbeddings(), persist_directory='chromadb4')

In [ ]:
db = chromadb.PersistentClient('./chromadb4')

In [80]:
query = ""
docs = db.similarity_search(query)
most_likely_document = docs[0].page_content

In [81]:
SEARCH_PROMPT = """
Im folgenden bekommst du mehrere Dokumente: <DOKUMENT1>, <DOKUMENT2> und <DOKUMENT3>.
Der Nutzer ist auf der Suche nach einem Rezept, welches du in einem der drei Dokumente findest.
Bestimme zunächst das passenste Rezept für die <REZEPTANFRAGE> des Nutzers.

Gib dann das Rezept in komprimerter Form zurück.
Dabei darfst du für drei Sätze in dem Grundrezept maximal zwei Sätze verwenden.
Achte darauf, dass du Mengenangaben und Temperaturen innerhalb der Schritte angibst.

Ausgabeformat:
```
<REZEPTNAME> (aus: <QUELLENNAME>)

Zutaten:
- <ZUTAT1>
- <ZUTAT2>
- <ZUTAT3>

Schritt 1: <SCHRITT1>
Schritt 2: <SCHRITT2>
Schritt 3: <SCHRITT3>
...
```

<DOKUMENT1>: {DOKUMENT1}
<QUELLE1>: {QUELLE1}
<DOKUMENT2>: {DOKUMENT2}
<QUELLE2>: {QUELLE2}
<DOKUMENT3>: {DOKUMENT3}
<QUELLE3>: {QUELLE3}
<REZEPTANFRAGE>: {REZEPTANFRAGE}

"""

In [82]:
p_template = PromptTemplate.from_template(SEARCH_PROMPT)

In [83]:
chat = OpenAIChat(temperature=0)
print(chat.predict(p_template.format(
    DOKUMENT1 = docs[0].page_content,
    DOKUMENT2 = docs[1].page_content,
    DOKUMENT3 = docs[2].page_content,
    REZEPTANFRAGE = query)))

Das passendste Rezept für die Anfrage nach Labskaus ist das Rezept "Küche der Nordseeinseln – Spezialitäten aus dem Meer" aus dem Dokument <DOKUMENT3>.

Hier ist das Rezept in komprimierter Form:

Tuffelschluffkes – Friesische Kartoffelpuffer

Zutaten:
- Kartoffeln
- Zwiebeln
- Eier

Schritt 1: Kartoffeln und Zwiebeln reiben und mit Eiern vermengen.
Schritt 2: Die Masse in einer Pfanne zu Kartoffelpuffern braten.
Schritt 3: Mit Beilagen wie Matjes oder Rote Bete servieren.
